In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Cinema_Audience_Forecasting_challenge/movie_theater_id_relation/movie_theater_id_relation.csv
/kaggle/input/Cinema_Audience_Forecasting_challenge/date_info/date_info.csv
/kaggle/input/Cinema_Audience_Forecasting_challenge/sample_submission/sample_submission.csv
/kaggle/input/Cinema_Audience_Forecasting_challenge/booknow_theaters/booknow_theaters.csv
/kaggle/input/Cinema_Audience_Forecasting_challenge/cinePOS_booking/cinePOS_booking.csv
/kaggle/input/Cinema_Audience_Forecasting_challenge/cinePOS_theaters/cinePOS_theaters.csv
/kaggle/input/Cinema_Audience_Forecasting_challenge/booknow_visits/booknow_visits.csv
/kaggle/input/Cinema_Audience_Forecasting_challenge/booknow_booking/booknow_booking.csv


# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Loading Data

In [3]:
visits = pd.read_csv('/kaggle/input/Cinema_Audience_Forecasting_challenge/booknow_visits/booknow_visits.csv')
book_booking = pd.read_csv('/kaggle/input/Cinema_Audience_Forecasting_challenge/booknow_booking/booknow_booking.csv')
cine_booking = pd.read_csv('/kaggle/input/Cinema_Audience_Forecasting_challenge/cinePOS_booking/cinePOS_booking.csv')
book_theaters = pd.read_csv('/kaggle/input/Cinema_Audience_Forecasting_challenge/booknow_theaters/booknow_theaters.csv')
cine_theaters = pd.read_csv('/kaggle/input/Cinema_Audience_Forecasting_challenge/cinePOS_theaters/cinePOS_theaters.csv')
theater_mapping = pd.read_csv('/kaggle/input/Cinema_Audience_Forecasting_challenge/movie_theater_id_relation/movie_theater_id_relation.csv')
date_info = pd.read_csv('/kaggle/input/Cinema_Audience_Forecasting_challenge/date_info/date_info.csv')
sample_submission = pd.read_csv('/kaggle/input/Cinema_Audience_Forecasting_challenge/sample_submission/sample_submission.csv')

print(f"Visits shape: {visits.shape}")
print(f"Sample submission shape: {sample_submission.shape}")

Visits shape: (214046, 3)
Sample submission shape: (38062, 2)


# Data Cleanup and Preprocessing

In [4]:
visits['show_date'] = pd.to_datetime(visits['show_date'])
book_booking['show_datetime'] = pd.to_datetime(book_booking['show_datetime'])
book_booking['booking_datetime'] = pd.to_datetime(book_booking['booking_datetime'])
cine_booking['show_datetime'] = pd.to_datetime(cine_booking['show_datetime'])
cine_booking['booking_datetime'] = pd.to_datetime(cine_booking['booking_datetime'])
date_info['show_date'] = pd.to_datetime(date_info['show_date'])

# Extract show_date from booking data
book_booking['show_date'] = book_booking['show_datetime'].dt.date
cine_booking['show_date'] = cine_booking['show_datetime'].dt.date

# Feature Engineering

In [5]:
# BookNow booking aggregations by theater and date
book_agg = book_booking.groupby(['book_theater_id', 'show_date']).agg({
    'tickets_booked': ['sum', 'mean', 'count', 'max']
}).reset_index()
book_agg.columns = ['book_theater_id', 'show_date', 'book_tickets_sum', 
                    'book_tickets_mean', 'book_booking_count', 'book_tickets_max']
book_agg['show_date'] = pd.to_datetime(book_agg['show_date'])

# CinePOS booking aggregations - map to BookNow IDs first
cine_booking_mapped = cine_booking.merge(theater_mapping, on='cine_theater_id', how='left')
cine_agg = cine_booking_mapped.groupby(['book_theater_id', 'show_date']).agg({
    'tickets_sold': ['sum', 'mean', 'count', 'max']
}).reset_index()
cine_agg.columns = ['book_theater_id', 'show_date', 'cine_tickets_sum',
                    'cine_tickets_mean', 'cine_booking_count', 'cine_tickets_max']
cine_agg['show_date'] = pd.to_datetime(cine_agg['show_date'])


# Merge Data

In [6]:
# Start with visits (training data)
df = visits.copy()

# Merge date info
df = df.merge(date_info, on='show_date', how='left')

# Clean book_theaters data
book_theaters_clean = book_theaters.dropna(subset=['book_theater_id'])

# Merge theater info (handle nulls in book_theater_id)
df = df.merge(book_theaters_clean[['book_theater_id', 'theater_type', 'theater_area']], 
              on='book_theater_id', how='left')

# Merge booking features
df = df.merge(book_agg, on=['book_theater_id', 'show_date'], how='left')
df = df.merge(cine_agg, on=['book_theater_id', 'show_date'], how='left')

# Fill missing booking features with 0
booking_cols = ['book_tickets_sum', 'book_tickets_mean', 'book_booking_count', 'book_tickets_max',
                'cine_tickets_sum', 'cine_tickets_mean', 'cine_booking_count', 'cine_tickets_max']
df[booking_cols] = df[booking_cols].fillna(0)

# Time Based Features

In [7]:
df['year'] = df['show_date'].dt.year
df['month'] = df['show_date'].dt.month
df['day'] = df['show_date'].dt.day
df['day_of_year'] = df['show_date'].dt.dayofyear
df['week_of_year'] = df['show_date'].dt.isocalendar().week
df['is_weekend'] = df['day_of_week'].isin(['Saturday', 'Sunday']).astype(int)
df['is_friday'] = (df['day_of_week'] == 'Friday').astype(int)
df['quarter'] = df['show_date'].dt.quarter

# Advanced time features
df['is_month_end'] = (df['show_date'].dt.day > 25).astype(int)
df['is_month_start'] = (df['show_date'].dt.day <= 5).astype(int)
df['season'] = (df['month'] % 12 + 3) // 3

# Days to weekend
df['days_to_weekend'] = df['day_of_week'].map({
    'Monday': 5, 'Tuesday': 4, 'Wednesday': 3, 'Thursday': 2,
    'Friday': 1, 'Saturday': 0, 'Sunday': 0
})

# Interaction features
df['weekend_x_month'] = df['is_weekend'] * df['month']
df['weekend_x_quarter'] = df['is_weekend'] * df['quarter']

# Lag Features and Theater Level Features

In [8]:
df = df.sort_values(['book_theater_id', 'show_date'])

# Add more lag periods
for lag in [1, 3, 7, 14, 21, 28]:
    df[f'audience_lag_{lag}'] = df.groupby('book_theater_id')['audience_count'].shift(lag)

# Rolling statistics
for window in [7, 14, 21, 28]:
    df[f'audience_roll_mean_{window}'] = df.groupby('book_theater_id')['audience_count'].transform(
        lambda x: x.rolling(window=window, min_periods=1).mean().shift(1)
    )
    df[f'audience_roll_std_{window}'] = df.groupby('book_theater_id')['audience_count'].transform(
        lambda x: x.rolling(window=window, min_periods=1).std().shift(1)
    )
    df[f'audience_roll_max_{window}'] = df.groupby('book_theater_id')['audience_count'].transform(
        lambda x: x.rolling(window=window, min_periods=1).max().shift(1)
    )

# Fill NaN lag features with median
lag_cols = [col for col in df.columns if 'lag_' in col or 'roll_' in col]
for col in lag_cols:
    df[col] = df[col].fillna(df[col].median())

# Average audience per theater
theater_avg = df.groupby('book_theater_id')['audience_count'].mean().reset_index()
theater_avg.columns = ['book_theater_id', 'theater_avg_audience']
df = df.merge(theater_avg, on='book_theater_id', how='left')

# Theater std
theater_std = df.groupby('book_theater_id')['audience_count'].std().reset_index()
theater_std.columns = ['book_theater_id', 'theater_std_audience']
df = df.merge(theater_std, on='book_theater_id', how='left')
df['theater_std_audience'] = df['theater_std_audience'].fillna(0)

# Encode Categorical Vairables

In [9]:
# Label encode categorical columns
from sklearn.preprocessing import LabelEncoder

cat_cols = ['book_theater_id', 'day_of_week', 'theater_type', 'theater_area']
label_encoders = {}

for col in cat_cols:
    if col in df.columns:
        le = LabelEncoder()
        df[col] = df[col].fillna('Unknown')
        df[col + '_encoded'] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le

# Prepare Training Data

In [10]:
# Select features for modeling
feature_cols = [
    'book_theater_id_encoded', 'day_of_week_encoded', 'theater_type_encoded', 
    'theater_area_encoded', 'year', 'month', 'day', 'day_of_year', 'week_of_year',
    'is_weekend', 'is_friday', 'quarter',
    'is_month_end', 'is_month_start', 'season', 'days_to_weekend',
    'weekend_x_month', 'weekend_x_quarter',
    'book_tickets_sum', 'book_tickets_mean', 'book_booking_count', 'book_tickets_max',
    'cine_tickets_sum', 'cine_tickets_mean', 'cine_booking_count', 'cine_tickets_max',
    'theater_avg_audience', 'theater_std_audience'
]

# Add lag features
feature_cols.extend(lag_cols)

# Remove any features not in df
feature_cols = [col for col in feature_cols if col in df.columns]

X = df[feature_cols].fillna(0)
y = df['audience_count']

print(f"Features: {len(feature_cols)}")
print(f"Training samples: {len(X)}")

Features: 46
Training samples: 214046


# Train Model

In [11]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time

# Split data (time-based split)
split_date = pd.to_datetime('2024-01-01')
train_mask = df['show_date'] < split_date
X_train, y_train = X[train_mask], y[train_mask]
X_val, y_val = X[~train_mask], y[~train_mask]

print(f"Train size: {len(X_train)}, Validation size: {len(X_val)}")

# Initialize models
models = {
    'Random Forest': RandomForestRegressor(
        n_estimators=300,
        max_depth=25,
        min_samples_split=5,
        min_samples_leaf=2,
        max_features='sqrt',
        random_state=42,
        n_jobs=-1
    ),
    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=300,
        max_depth=10,
        learning_rate=0.05,
        subsample=0.8,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42
    ),
    'Extra Trees': ExtraTreesRegressor(
        n_estimators=300,
        max_depth=25,
        min_samples_split=5,
        min_samples_leaf=2,
        max_features='sqrt',
        random_state=42,
        n_jobs=-1
    )
}

# Store results
results = []
trained_models = {}

# Train and evaluate each model
for model_name, model in models.items():
    print(f"\n{'='*60}")
    print(f"Training {model_name}...")
    print(f"{'='*60}")
    
    # Train
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    # Store trained model
    trained_models[model_name] = model
    
    # Predict
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    
    # Ensure non-negative predictions
    y_pred_train = np.maximum(y_pred_train, 0)
    y_pred_val = np.maximum(y_pred_val, 0)
    
    # Calculate metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    val_rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    train_mae = mean_absolute_error(y_train, y_pred_train)
    val_mae = mean_absolute_error(y_val, y_pred_val)
    train_r2 = r2_score(y_train, y_pred_train)
    val_r2 = r2_score(y_val, y_pred_val)
    
    # Store results
    results.append({
        'Model': model_name,
        'Train RMSE': train_rmse,
        'Val RMSE': val_rmse,
        'Train MAE': train_mae,
        'Val MAE': val_mae,
        'Train R²': train_r2,
        'Val R²': val_r2,
        'Training Time (s)': training_time
    })
    
    print(f"Training Time: {training_time:.2f}s")
    print(f"Train RMSE: {train_rmse:.2f} | Val RMSE: {val_rmse:.2f}")
    print(f"Train MAE: {train_mae:.2f} | Val MAE: {val_mae:.2f}")
    print(f"Train R²: {train_r2:.4f} | Val R²: {val_r2:.4f}")

# Model comparison
print("\n" + "="*80)
print("📊 MODEL COMPARISON SUMMARY")
print("="*80)

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

# Find best model
best_model_name = results_df.loc[results_df['Val RMSE'].idxmin(), 'Model']
print(f"\n🏆 BEST MODEL: {best_model_name} (Lowest Validation RMSE)")

Train size: 174535, Validation size: 39511

Training Random Forest...
Training Time: 79.90s
Train RMSE: 13.31 | Val RMSE: 21.05
Train MAE: 8.39 | Val MAE: 14.84
Train R²: 0.8396 | Val R²: 0.5355

Training Gradient Boosting...
Training Time: 1002.75s
Train RMSE: 15.27 | Val RMSE: 21.87
Train MAE: 11.21 | Val MAE: 15.09
Train R²: 0.7891 | Val R²: 0.4988

Training Extra Trees...
Training Time: 33.86s
Train RMSE: 15.17 | Val RMSE: 21.15
Train MAE: 10.18 | Val MAE: 14.90
Train R²: 0.7919 | Val R²: 0.5311

📊 MODEL COMPARISON SUMMARY
            Model  Train RMSE  Val RMSE  Train MAE   Val MAE  Train R²   Val R²  Training Time (s)
    Random Forest   13.314207 21.052830   8.387804 14.843346  0.839630 0.535497          79.896403
Gradient Boosting   15.266554 21.867660  11.214571 15.092368  0.789150 0.498845        1002.752709
      Extra Trees   15.165488 21.153316  10.177130 14.900582  0.791932 0.531052          33.859308

🏆 BEST MODEL: Random Forest (Lowest Validation RMSE)


# Ensemble Predictions

In [12]:
ensemble_weights = {
    'Random Forest': 0.5,
    'Gradient Boosting': 0.3,
    'Extra Trees': 0.2
}

print(f"Ensemble weights: {ensemble_weights}")

Ensemble weights: {'Random Forest': 0.5, 'Gradient Boosting': 0.3, 'Extra Trees': 0.2}


# Preparing Test Data

In [13]:
# Extract theater_id and date from submission ID
sample_submission['book_theater_id'] = sample_submission['ID'].str.split('_').str[:2].str.join('_')
sample_submission['show_date'] = pd.to_datetime(sample_submission['ID'].str.split('_').str[2])

test_df = sample_submission[['book_theater_id', 'show_date']].copy()

# Merge date info
test_df = test_df.merge(date_info, on='show_date', how='left')

# For dates not in date_info, infer day_of_week
if test_df['day_of_week'].isna().any():
    test_df['day_of_week'] = test_df['day_of_week'].fillna(
        test_df['show_date'].dt.day_name()
    )

# Merge theater info
test_df = test_df.merge(book_theaters_clean[['book_theater_id', 'theater_type', 'theater_area']], 
                       on='book_theater_id', how='left')

# Merge booking features
test_df = test_df.merge(book_agg, on=['book_theater_id', 'show_date'], how='left')
test_df = test_df.merge(cine_agg, on=['book_theater_id', 'show_date'], how='left')

# Fill missing booking features
test_df[booking_cols] = test_df[booking_cols].fillna(0)

# Time features
test_df['year'] = test_df['show_date'].dt.year
test_df['month'] = test_df['show_date'].dt.month
test_df['day'] = test_df['show_date'].dt.day
test_df['day_of_year'] = test_df['show_date'].dt.dayofyear
test_df['week_of_year'] = test_df['show_date'].dt.isocalendar().week
test_df['is_weekend'] = test_df['day_of_week'].isin(['Saturday', 'Sunday']).astype(int)
test_df['is_friday'] = (test_df['day_of_week'] == 'Friday').astype(int)
test_df['quarter'] = test_df['show_date'].dt.quarter

# Advanced time features (SAME AS TRAINING)
test_df['is_month_end'] = (test_df['show_date'].dt.day > 25).astype(int)
test_df['is_month_start'] = (test_df['show_date'].dt.day <= 5).astype(int)
test_df['season'] = (test_df['month'] % 12 + 3) // 3
test_df['days_to_weekend'] = test_df['day_of_week'].map({
    'Monday': 5, 'Tuesday': 4, 'Wednesday': 3, 'Thursday': 2,
    'Friday': 1, 'Saturday': 0, 'Sunday': 0
})
test_df['days_to_weekend'] = test_df['days_to_weekend'].fillna(0)
test_df['weekend_x_month'] = test_df['is_weekend'] * test_df['month']
test_df['weekend_x_quarter'] = test_df['is_weekend'] * test_df['quarter']

# Theater average
test_df = test_df.merge(theater_avg, on='book_theater_id', how='left')
test_df['theater_avg_audience'] = test_df['theater_avg_audience'].fillna(
    df['audience_count'].median()
)

# Theater std
test_df = test_df.merge(theater_std, on='book_theater_id', how='left')
test_df['theater_std_audience'] = test_df['theater_std_audience'].fillna(0)

# CREATE LAG FEATURES FOR TEST
print("Creating lag features for test data...")

# Combine train and test to properly compute lags
combined_df = pd.concat([
    df[['book_theater_id', 'show_date', 'audience_count']],
    test_df[['book_theater_id', 'show_date']].assign(audience_count=np.nan)
], ignore_index=True).sort_values(['book_theater_id', 'show_date'])

# Create lag features on combined data (match training lags)
for lag in [1, 3, 7, 14, 21, 28]:
    combined_df[f'audience_lag_{lag}'] = combined_df.groupby('book_theater_id')['audience_count'].shift(lag)

# Rolling statistics
for window in [7, 14, 21, 28]:
    combined_df[f'audience_roll_mean_{window}'] = combined_df.groupby('book_theater_id')['audience_count'].transform(
        lambda x: x.rolling(window=window, min_periods=1).mean().shift(1)
    )
    combined_df[f'audience_roll_std_{window}'] = combined_df.groupby('book_theater_id')['audience_count'].transform(
        lambda x: x.rolling(window=window, min_periods=1).std().shift(1)
    )
    combined_df[f'audience_roll_max_{window}'] = combined_df.groupby('book_theater_id')['audience_count'].transform(
        lambda x: x.rolling(window=window, min_periods=1).max().shift(1)
    )

# Get only the lag columns for test data
test_lag_data = combined_df[combined_df['audience_count'].isna()][
    ['book_theater_id', 'show_date'] + lag_cols
].reset_index(drop=True)

# Merge lag features into test_df
test_df = test_df.merge(
    test_lag_data,
    on=['book_theater_id', 'show_date'],
    how='left'
)

# Fill remaining NaNs with training medians
for col in lag_cols:
    if col in test_df.columns:
        test_df[col] = test_df[col].fillna(df[col].median())

# Encode categorical features
for col in cat_cols:
    if col in test_df.columns:
        test_df[col] = test_df[col].fillna('Unknown')
        test_df[col + '_encoded'] = test_df[col].astype(str).map(
            lambda x: label_encoders[col].transform([x])[0] 
            if x in label_encoders[col].classes_ else -1
        )
        test_df[col + '_encoded'] = test_df[col + '_encoded'].fillna(-1)

# Select features
X_test = test_df[feature_cols].fillna(0)

print(f"Test samples: {len(X_test)}")

Creating lag features for test data...
Test samples: 38062


# Make Predictions

In [14]:
# Get predictions from all models
all_predictions = []
for model_name, model in trained_models.items():
    pred = model.predict(X_test)
    pred = np.maximum(pred, 0)
    all_predictions.append(pred * ensemble_weights[model_name])
    print(f"{model_name} predictions - Mean: {pred.mean():.2f}, Range: [{pred.min():.0f}, {pred.max():.0f}]")

# Sum weighted predictions
predictions = np.sum(all_predictions, axis=0)
predictions = np.maximum(predictions, 0)
predictions = np.round(predictions).astype(int)

print(f"\nFinal ensemble predictions:")
print(f"  Mean: {predictions.mean():.2f}")
print(f"  Median: {np.median(predictions):.2f}")
print(f"  Range: [{predictions.min()}, {predictions.max()}]")

Random Forest predictions - Mean: 39.77, Range: [4, 142]
Gradient Boosting predictions - Mean: 41.47, Range: [2, 225]
Extra Trees predictions - Mean: 39.45, Range: [4, 134]

Final ensemble predictions:
  Mean: 40.22
  Median: 37.00
  Range: [4, 159]


# Submission

In [15]:
submission = pd.DataFrame({
    'ID': sample_submission['ID'],
    'audience_count': predictions
})

# Save submission
submission.to_csv('submission.csv', index=False)
print("\n✅ Submission file saved as 'submission.csv'")


✅ Submission file saved as 'submission.csv'
